In [1]:
#inputs
input_file = 'globalTestFormat.csv' #the name of the input file

mrt_title = 'ACCEPT 2.0: A compilation of X-ray gas properties for clusters of galaxies observed by the Chandra X-ray Observatory' #the title of the paper
mrt_authors = 'Donahue M., Baldi A., Frisbie R., Koeppe D., Voit G.M.' #the authors of the paper
mrt_table = 'Global Properties' #name of the table


RA_column = '8' #MUST BE DECIMAL DEGREES
DEC_column = '9' #MUST BE DECIMAL DEGREES

In [3]:
from astropy.table import Table
import pandas as pd
import numpy as np
import tempfile
import os
import shutil

#important bit about checked units: https://journals.aas.org/mrt-labels/#checked

file_loc = r'../data/'+input_file
raw_name = input_file[:-4]

if not os.path.exists(r'table_generate_output/'+raw_name):
    os.makedirs(r'table_generate_output/'+raw_name)
output_path = r'table_generate_output/'+raw_name+r'/'

df = pd.read_csv(file_loc)
unitmap = df.to_dict(orient='records')[0] #create a map for the units of each column
descriptionmap = df.to_dict(orient='records')[1] #create a map for the description of each column
df = df.drop([0]) #drops the units row
df = df.drop([1]) #drops the description row

tempcsv = tempfile.NamedTemporaryFile() #writes a temporary csv file to be read by astropy, this fixes an issue where directly feeding astropy a pandas df doesn't correctly assign datatypes
df.to_csv(tempcsv,index=False)

csv = Table.read(tempcsv,format='csv',units=unitmap,descriptions=descriptionmap) #read the csv into an astropy table

csv.write(output_path+raw_name+'.dat', overwrite=True, format='mrt') #uses astropy to write and mrt lacking metadata

mrt_replaceables = { #used to replace empty metadata in the mrt
    'Title:': 'Title: ' + mrt_title,
    'Authors:': 'Authors: '+ mrt_authors,
    'Table:': 'Table: '+ mrt_table,
    'table.dat': raw_name+'.dat',
}

with open(output_path+raw_name+'.dat', 'r+') as file: #replaces empty metadata
    read_content=file.read()
    for key in mrt_replaceables:
        read_content=read_content.replace(key, mrt_replaceables[key])
    file.seek(0)
    file.write(read_content)

pd.read_csv(file_loc,header=[0,1],skiprows=[2]).to_html(output_path+raw_name+'.html',index=False) #makes the html table via pandas


# consearch creation

shutil.copy('search.php', output_path)
shutil.copy(file_loc, output_path)

consearch_replaceables = { #used to fill in info for conesearch script
    'file_name_replace': '"' + input_file + '"',
    'RA_index_replace': RA_column,
    'DEC_index_replace': DEC_column
}

with open(output_path+'search.php', 'r+') as file: #fills in necessary info for conesearch script
    read_content=file.read()
    for key in consearch_replaceables:
        read_content=read_content.replace(key, consearch_replaceables[key])
    file.seek(0)
    file.write(read_content)